
# 📄 CSV to YAML Protocol Converter

This tool converts a CSV file containing lab protocol supplies into a properly formatted YAML file for running in the Inventory Checker.

Follow the instructions below to ensure your file is in the correct format and to run the conversion.

---

## CSV File Format

Your CSV file **must** contain the following four columns in any order:

* `Category`: Specifies where the supply is used. This must be either **`per_sample`** or **`per_plate`**.
* `Item`: The name or description of the supply item. This **must** be copied exaclty from the inventory sheet.
* `Quantity`: The numerical quantity of the item needed.
* `Unit`: The unit of measurement for the quantity (`ul`, `tips`, `item`). If a different unit is required, contact Aubrie (aubrie.onoufriou@noaa.gov) to do this.

#### Example CSV Structure:
| Category | Item | Quantity | Unit |
| :--- | :--- | :--- | :--- |
| per_sample | 200 ul Ranin Tips (960 tips) | 4 | tips |
| per_sample | ddPCR Supermix for Probes (5 mL) | 10 | ul |
| per_plate | ddPCR 96 well Semi skirted plates | 1 | item |

---

## How to Use This Tool

1.  **Prepare Your Data**: Make sure your supply list is saved as a **CSV file** that matches the format described above.
2.  **Run the Code**: Execute the code cell by pressing **Run all** above. A blue **"Upload New CSV to Begin"** button will appear.
3.  **Upload Your File**: Click the button and select the CSV file from your computer.
4.  **Name the Protocol**: This name will appear inside the yaml file and be used when the new yaml file is downloaded.
5.  **Generate the yaml file and view it**: Click the **"Generate & Preview"** button to see what the yaml file will look like.
6.  **Download the Output**: If happy with the preview, click **"Download YAML"** to download the new yaml file to your computer.


In [ ]:
# @title CSV to YAML Protocol Converter

# Step 1: Import all necessary libraries
import pandas as pd
import yaml
from google.colab import files
import io
import os
import re
import ipywidgets as widgets
from IPython.display import display, clear_output

# Step 2: Create the main "start" button and a container for the rest of the UI
upload_button = widgets.Button(
    description="Upload New CSV to Begin",
    button_style='primary',
    icon='upload',
    layout=widgets.Layout(width='50%', height='50px')
)

# This container will hold the widgets for each file processing task
sub_container = widgets.VBox([])

# This is the main function that runs every time you want to process a new file
def start_new_upload(b):
    # Clear the container from the previous run
    sub_container.children = []

    # Use a separate output widget to ask for the file
    upload_prompt_area = widgets.Output()

    # Display the container that will hold the upload prompt
    sub_container.children = [upload_prompt_area]

    uploader = None # Initialize uploader to None
    with upload_prompt_area:
        try:
            print("Please upload your CSV file using the button below:")
            uploader = files.upload()
        except Exception as e:
            # This block catches the error if the user cancels the upload
            clear_output(wait=True)
            print("❌ Upload cancelled or failed.")

    # If upload was cancelled or failed, stop here
    if not uploader:
        return

    # If upload is successful, create the rest of the interface
    input_filename = next(iter(uploader))
    default_protocol_name = os.path.splitext(input_filename)[0]

    notes_widget = widgets.Textarea(
        value='',
        placeholder='Enter any notes here. They will be added as comments to the YAML file.',
        description='Optional Notes:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='70%', height='100px')
    )
    protocol_name_widget = widgets.Text(value=default_protocol_name, description='Protocol Name:', style={'description_width': 'initial'}, layout=widgets.Layout(width='70%'))
    generate_button = widgets.Button(description="Generate & Preview", button_style='info', icon='cogs')
    download_button = widgets.Button(description="Download YAML", button_style='success', icon='download', disabled=True)
    output_area = widgets.Output()

    def generate_and_preview(b):
        with output_area:
            output_area.clear_output()
            df = pd.read_csv(io.BytesIO(uploader[input_filename]))
            for col in df.select_dtypes(['object']):
                df[col] = df[col].str.strip()

            protocol_name_from_widget = protocol_name_widget.value
            sanitized_filename = re.sub(r'[^\w-]', '', protocol_name_from_widget.replace(' ', '_'))

            global output_filename
            output_filename = f"{sanitized_filename}.yaml"

            protocol_data = {
                'protocol_name': protocol_name_from_widget, 'supplies_per_sample': [], 'supplies_per_plate': []
            }
            for _, row in df.iterrows():
                supply_item = {'item': row['Item'], 'quantity': row['Quantity'], 'unit': row['Unit']}
                if row['Category'] == 'per_sample':
                    protocol_data['supplies_per_sample'].append(supply_item)
                elif row['Category'] == 'per_plate':
                    protocol_data['supplies_per_plate'].append(supply_item)

            notes_text = notes_widget.value
            commented_notes = ""
            if notes_text.strip():
                commented_notes = "\n".join([f"# {line}" for line in notes_text.split('\n')]) + "\n---\n"

            main_yaml_content = yaml.dump(protocol_data, sort_keys=False, default_flow_style=False)

            with open(output_filename, 'w') as yaml_file:
                yaml_file.write(commented_notes)
                yaml_file.write(main_yaml_content)

            print(f"✅ Preview generated for '{output_filename}'!")
            print("\n" + "="*20 + "\n  YAML File Preview\n" + "="*20)
            with open(output_filename, 'r') as f:
                print(f.read())
            print("="*20 + "\nFile is ready. You can now click the download button.")

            download_button.description = f"Download {output_filename}"
            download_button.disabled = False

    def download_file(b):
        files.download(output_filename)

    generate_button.on_click(generate_and_preview)
    download_button.on_click(download_file)

    # Assemble the new interface and display it in the container
    button_box = widgets.HBox([generate_button, download_button])
    instructions = widgets.HTML(value=f"<hr><b>File '{input_filename}' loaded.</b><br><b>1. Add any optional notes.</b><br><b>2. Confirm the protocol name.</b><br><b>3. Click 'Generate & Preview', then download.</b>")
    sub_container.children = [instructions, notes_widget, protocol_name_widget, button_box, output_area]

# Link the main button to the function that starts the process
upload_button.on_click(start_new_upload)

# Display the initial button and the container that will hold everything else
display(upload_button, sub_container)